In [ ]:
pip install pandas scikit-learn

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive


'AI Project'
 Chitosan.asc
'cloud essential knowledge based.pdf'
'Colab Notebooks'
 convertio.co
 cropper
 machineLearning
'Notion Material'
'oke 1'
'PENGUMUMAN NEC UNESA 2023.gdoc'
 Picture1.png
 Picture2.png
'RNN Citra Minyak'
'Screenshot_2023-10-17-17-34-05-880_com (1).instagram.android.jpg'
 Screenshot_2023-10-17-17-34-05-880_com.instagram.android.jpg
'Screenshot (533).png'
'Screenshot (535).png'
'Some Project'
'WhatsApp Image 2024-04-04 at 19.48.38_3e8205cd.jpg'
'WhatsApp Image 2024-04-04 at 19.49.32_2bdd8762.jpg'


In [ ]:
! pip install accelerate -U
! pip install tokenizers
! pip install transformers datasets evaluate
!CUDA_LAUNCH_BLOCKING=1

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00


In [1]:
from datasets import load_dataset

poetry = load_dataset("aditdwi123/cyber-bullying-dataset")

print(poetry)

ModuleNotFoundError: No module named 'datasets'

In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/machineLearning/CB_detection/id-cyberbullying-instagram.tsv', sep='\t', encoding='ISO-8859-1')

print(data.head())

   no                                               text label
0   1                    Klo fadli zon goblok, boleeeh?     CB
1   2  @detikcom Jangan nyinyirin pak fadli zon, dia ...    CB
2   3  @ahmadwaluy @bravo1282 @fadlizon Kritik siapa ...    CB
3   4   @fadlizon @jokowi Hati penuh dendam,  pikiran...    CB
4   5  @fadlizon @jokowi Kerja anda apa zon bisanya c...    CB


In [ ]:
import re
import string
from sklearn.model_selection import train_test_split

def clean_text(text):
    # Menghapus URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Menghapus mention (@username) dan hashtag (#hashtag)
    text = re.sub(r'\@\w+|\#','', text)
    # Menghapus angka
    text = re.sub(r'\d+', '', text)
    # Menghapus tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Mengubah ke huruf kecil
    text = text.lower()
    return text

poetry['cleaned_text'] = poetry['kalimat'].apply(clean_text)

print(poetry[['kalimat', 'cleaned_text']].head())


KeyError: 'kalimat'

In [ ]:
data['label'] = data['label'].map({'CB': 1, 'Non_CB': 0})

print(data['label'].value_counts())


label
1    150
0    150
Name: count, dtype: int64


In [ ]:
X = data['cleaned_text']
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)

X_train_vec = vectorizer.fit_transform(X_train)

X_test_vec = vectorizer.transform(X_test)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(X_train_vec, y_train)


LogisticRegression()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Melakukan prediksi pada data pengujian
y_pred = model.predict(X_test_vec)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi: {accuracy:.2f}")

# Menampilkan laporan klasifikasi
print(classification_report(y_test, y_pred))


Akurasi: 0.80
              precision    recall  f1-score   support

           0       0.79      0.84      0.81        31
           1       0.81      0.76      0.79        29

    accuracy                           0.80        60
   macro avg       0.80      0.80      0.80        60
weighted avg       0.80      0.80      0.80        60



In [ ]:
import numpy as np

# Mendapatkan skor probabilitas
y_pred_proba = model.predict_proba(X_test_vec)[:, 1]  # Probabilitas kelas 'CB'

# Menampilkan teks, prediksi, dan skor kepercayaan
for i in range(len(X_test)):
    print(f"Teks: {X_test.iloc[i]}")
    print(f"Prediksi: {'CB' if y_pred[i] == 1 else 'Non_CB'} dengan Kepercayaan: {y_pred_proba[i]*100:.2f}%")
    print("-" * 50)


Teks:  jokowi lebih dewasa pak jokowi gak nanggapin omongan si fadli zon 
Prediksi: Non_CB dengan Kepercayaan: 44.39%
--------------------------------------------------
Teks:    benar tapi yang menimbulkan tanda tanya mengapakah menteri keuangan harus sidak barang sitaan ilegal  apa harus dihitung potensi perolehan uang yang sayang untuk dibuang saya rasa tidak masalah narkoba diperdagangkan asalkan bisa masuk pajak negara 
Prediksi: Non_CB dengan Kepercayaan: 40.53%
--------------------------------------------------
Teks:   kalau gak ada penjaga tidak mungkin di tangkap pak 
Prediksi: Non_CB dengan Kepercayaan: 42.34%
--------------------------------------------------
Teks:    fadli zon bukan mengkritiktapi mulut rombengan alias nyablak mulu 
Prediksi: CB dengan Kepercayaan: 62.07%
--------------------------------------------------
Teks:          kl mslh coblos mencoblos itu hak masing mas  kalo saya tetap setia dgn apa yg saya yakini  tetap bangga berada di barisan pendukung prabowo 

In [ ]:
import pickle

# Path penyimpanan di Google Drive
save_path = '/content/drive/MyDrive/machineLearning/CB_detection/model_logistic_regression.pkl'

# Menyimpan model menggunakan pickle
with open(save_path, 'wb') as file:
    pickle.dump(model, file)

print(f"Model berhasil disimpan di {save_path}")


Model berhasil disimpan di /content/drive/MyDrive/machineLearning/CB_detection/model_logistic_regression.pkl


In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords

# Unduh daftar stopwords jika belum ada (jalankan hanya satu kali)
import nltk
nltk.download('stopwords')

# Memuat dataset
data = pd.read_csv('/content/drive/MyDrive/machineLearning/CB_detection/id-cyberbullying-instagram.tsv', sep='\t', encoding='latin-1')


import string


def clean_text(text):
    # Menghapus URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Menghapus mention (@username) dan hashtag (#hashtag)
    text = re.sub(r'\@\w+|\#','', text)
    # Menghapus angka
    text = re.sub(r'\d+', '', text)
    # Menghapus tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Mengubah ke huruf kecil
    text = text.lower()
    return text

data['cleaned_text'] = data['text'].apply(clean_text)

print(data[['text', 'cleaned_text']].head())

# Mengonversi label ke format numerik
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Konversi ke numerik

# Membagi dataset menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Menggunakan TF-IDF untuk mengubah teks menjadi fitur
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Mendefinisikan model
models = {
    'Logistic Regression': LogisticRegression(max_iter=200),
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    'Neural Network': MLPClassifier(max_iter=500)
}

# Melatih dan mengevaluasi setiap model
results = {}

for model_name, model in models.items():
    model.fit(X_train_vec, y_train)  # Melatih model
    y_pred = model.predict(X_test_vec)  # Memprediksi pada data test
    accuracy = accuracy_score(y_test, y_pred)  # Menghitung akurasi
    results[model_name] = {
        'accuracy': accuracy,
        'classification_report': classification_report(y_test, y_pred, target_names=label_encoder.classes_)
    }

# Menampilkan hasil evaluasi
for model_name, result in results.items():
    print(f"{model_name} Accuracy: {result['accuracy']:.4f}")
    print(result['classification_report'])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                text  \
0                    Klo fadli zon goblok, boleeeh?    
1  @detikcom Jangan nyinyirin pak fadli zon, dia ...   
2  @ahmadwaluy @bravo1282 @fadlizon Kritik siapa ...   
3   @fadlizon @jokowi Hati penuh dendam,  pikiran...   
4  @fadlizon @jokowi Kerja anda apa zon bisanya c...   

                                        cleaned_text  
0                      klo fadli zon goblok boleeeh   
1   jangan nyinyirin pak fadli zon dia gitu punya...  
2     kritik siapa bisa di penjara fadli zon krit...  
3     hati penuh dendam  pikiran selalu negatif m...  
4    kerja anda apa zon bisanya cuma nyinyi r aja...  


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:31:56] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Logistic Regression Accuracy: 0.7111
              precision    recall  f1-score   support

          CB       0.79      0.62      0.70        48
      Non_CB       0.65      0.81      0.72        42

    accuracy                           0.71        90
   macro avg       0.72      0.72      0.71        90
weighted avg       0.73      0.71      0.71        90

SVM Accuracy: 0.6778
              precision    recall  f1-score   support

          CB       0.74      0.60      0.67        48
      Non_CB       0.63      0.76      0.69        42

    accuracy                           0.68        90
   macro avg       0.69      0.68      0.68        90
weighted avg       0.69      0.68      0.68        90

Random Forest Accuracy: 0.6222
              precision    recall  f1-score   support

          CB       0.73      0.46      0.56        48
      Non_CB       0.57      0.81      0.67        42

    accuracy                           0.62        90
   macro avg       0.65      0.63      